In [5]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


pdf = "Error_PDFS/ARP_09_25_2022.pdf"
#pdf = "Example_PDFS/equibaseFile.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 3

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_25_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


track : Arapahoe Park
date : September 25, 2022
race_num : 4
criteria : VIP S
track_length :  One And One Sixteenth Miles On The Dirt
purse : $26,550
weather : Clear
track_type : Fast
off_time : 2:16
start : Good for all
race_type : stakes
Looking for tables...
Found table top bound at 150.7059999999999
Found table bottom bound at 262.00600000000003
Looking for tables...
Found table top bound at 430.456
Found table bottom bound at 551.61


In [4]:
result_tables[0]

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,Start,1/4,1/2,3/4,Str,...,Comments,horse_name,Last Raced_super_script,1/4_length_behind,1/2_length_behind,3/4_length_behind,Str_length_behind,Fin_length_behind,Wgt,M/E
0,5Sep22 ARP,5.0,Adrian,Ramos,5.0,3.0,7,7,5,1,...,3wd bid 2nd tur,LIKE MOULIN ROUGE,1 2,1 1/2,1/2,1/2,Head,4,124,b
1,30Jul22 RET,6.0,Hugo,Herrera,6.0,4.0,3,3,2,2,...,stalked 2pth out,PADDYS DAY,1 2,1/2,1,1,1/2,Neck,124,L
2,30Jul22 RET,9.0,Francisco,Giles,9.0,7.0,8,8,6,3,...,circled field six,QUICK SAND AA,1 1,2,8,1/2,Head,1 3/4,124,BL b
3,17Sep22 LS,7.0,Bryan,McNeil,7.0,5.0,6,6,7,6,...,inside trip mild b,RB RATHOWAYNE,1 4,1/2,1/2,1/2,1,Head,124,L b
4,11Sep22 ARP,3.0,David,Martin,3.0,6.0,4,4,3,5,...,rail trip no rally,UPTOWN RINGOFFIRE,1 3,1/2,1,Head,2,5 3/4,124,L b
5,5Sep22 ARP,2.0,Travis,Wales,2.0,2.0,5,5,4,4,...,4wd entering str,SAND CANYON JS,1 1,1/2,Head,1,1/2,9 3/4,124,BL bf
6,19Aug22 RET,10.0,Pedro,Gonzalez,10.0,8.0,2,2,8,8,...,3wd up close fa,JOB AA,3 5,1/2,Head,10,20,24,124,L b
7,5Sep22 ARP,1.0,Nathan,Haar,1.0,1.0,1,1,1,7,...,inside speed sto,PROUD AMERICAN,1 3,1,1/2,1/2,6,19,124,BL f
8,11Sep22 ARP,8.0,Daniel,Aguilar,8.0,10.0,9,9,9,9,...,seven wide first,BREAD LINE,1 5,3,2,4,6,7 3/4,124,BL b
9,29Aug22 ARP,4.0,Mike,Luark,4.0,9.0,10,10,10,10,...,trailed tough foe,AA TWO FACE,1 5,NaN,NaN,NaN,NaN,NaN,124,L


In [6]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    print(header)
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0].astype(object)
    bottom_table = result_tables[1].astype(object)
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_25_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 5}, {'page_num': 1, 'horse_count': 9}, {'page_num': 2, 'horse_count': 6}, {'page_num': 3, 'horse_count': 10}, {'page_num': 4, 'horse_count': 6}, {'page_num': 5, 'horse_count': 8}, {'page_num': 6, 'horse_count': 7}, {'page_num': 7, 'horse_count': 9}, {'page_num': 8, 'horse_count': 6}]
{'track': 'Arapahoe Park', 'date': 'September 25, 2022', 'race_num': '1', 'criteria': 'FOR MAIDENS, TWO YEARS OLD', 'track_length': ' Four Hundred Yards On The Dirt', 'purse': '$10,300', 'weather': 'Clear', 'track_type': 'Fast', 'off_time': '1:01', 'start': 'Good for all', 'race_type': 'maiden'}
Looking for tables...
Found table top bound at 126.14599999999996
Found table bottom bound at 186.54599999999994
Looking for tables...
Found table top bound at 308.471


The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_25_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 381.665
    Last Raced  Pgm jockey_first_name  jockey_last_name   PP Start Str 1 Str  \
0  29Aug22 ARP  3.0            Ramiro            Garcia  3.0   3.0     2   2   
1  28Aug22 ARP  2.0         Francisco             Giles  2.0   2.0     1   1   
2  14Aug22 ARP  4.0              Juan  Rosales Gallegos  4.0   1.0     3   3   
3  29Aug22 ARP  1.0           Jhaciel              Soto  1.0   4.0     4   4   
4  14Aug22 ARP  5.0               Rey         Hernandez  5.0   5.0     5   5   

  Fin   Odds  ...                date race_num                    criteria  \
0   1   6.00  ...  September 25, 2022        1  FOR MAIDENS, TWO YEARS OLD   
1   2  0.60*  ...  September 25, 2022        1  FOR MAIDENS, TWO YEARS OLD   
2   3   8.60  ...  September 25, 2022        1  FOR MAIDENS, TWO YEARS OLD   
3   4   2.40  ...  September 25, 2022        1  FOR MAIDENS, TWO YEARS OLD   
4   5  12.80  ...  September 25, 2022        1  FOR MAIDENS, TWO YEARS OLD   

              

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_25_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

Looking for tables...
Found table top bound at 115.41399999999999
Found table bottom bound at 226.71399999999994
Looking for tables...
Found table top bound at 376.642


C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_25_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 481.276
     Last Raced   Pgm jockey_first_name  jockey_last_name    PP Start Str 1  \
0   29Aug22 ARP   3.0            Ramiro            Garcia   3.0   3.0     2   
1   28Aug22 ARP   2.0         Francisco             Giles   2.0   2.0     1   
2   14Aug22 ARP   4.0              Juan  Rosales Gallegos   4.0   1.0     3   
3   29Aug22 ARP   1.0           Jhaciel              Soto   1.0   4.0     4   
4   14Aug22 ARP   5.0               Rey         Hernandez   5.0   5.0     5   
5    3Sep22 IND   6.0           Valente           Vorrath   NaN   2.0   NaN   
6   29Aug22 ARP   5.0              Hugo           Herrera   NaN   1.0   NaN   
7   29Aug22 ARP   7.0   Juan Manuel Ros          Gallegos   NaN   6.0   NaN   
8   14Aug22 ARP   9.0             David            Valles   NaN   3.0   NaN   
9   29Aug22 ARP   2.0            Martin           Lazalde   NaN   5.0   NaN   
10   4Sep22 ARP   3.0            Ramiro            Garcia   NaN   9.0   NaN   
11  14Aug22 ARP 

Looking for tables...
Found table top bound at 116.53800000000001
Found table bottom bound at 197.83799999999997
Looking for tables...
Found table top bound at 459.288
Found table bottom bound at 543.9219999999999
     Last Raced  Pgm jockey_first_name  jockey_last_name   PP Start Str 1 Str  \
0   29Aug22 ARP  3.0            Ramiro            Garcia  3.0   3.0     2   2   
1   28Aug22 ARP  2.0         Francisco             Giles  2.0   2.0     1   1   
2   14Aug22 ARP  4.0              Juan  Rosales Gallegos  4.0   1.0     3   3   
3   29Aug22 ARP  1.0           Jhaciel              Soto  1.0   4.0     4   4   
4   14Aug22 ARP  5.0               Rey         Hernandez  5.0   5.0     5   5   
..          ...  ...               ...               ...  ...   ...   ...  ..   
61  11Sep22 ARP  4.0            Adrian             Ramos  4.0   6.0   NaN   5   
62   6Sep22 ARP  6.0              Hugo           Herrera  5.0   1.0   NaN   1   
63   6Sep22 ARP  2.0             Pedro          Gonzalez 

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [7]:
pdf_df

,Last Raced,Pgm,jockey_first_name,jockey_last_name,PP,Start,Str 1,Str,Fin,Odds,...,1/2_length_behind_RLP,3/4_length_behind_RLP,3/16,3/8,3/16_length_behind,3/8_length_behind,3/16_RLP,3/8_RLP,3/16_length_behind_RLP,3/8_length_behind_RLP
0,29Aug22 ARP,3.0,Ramiro,Garcia,3.0,3.0,2,2,1,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28Aug22 ARP,2.0,Francisco,Giles,2.0,2.0,1,1,2,0.60*,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14Aug22 ARP,4.0,Juan,Rosales Gallegos,4.0,1.0,3,3,3,8.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,29Aug22 ARP,1.0,Jhaciel,Soto,1.0,4.0,4,4,4,2.40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14Aug22 ARP,5.0,Rey,Hernandez,5.0,5.0,5,5,5,12.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,11Sep22 ARP,4.0,Adrian,Ramos,4.0,6.0,NaN,5,2,7.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,6Sep22 ARP,6.0,Hugo,Herrera,5.0,1.0,NaN,1,3,4.90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,6Sep22 ARP,2.0,Pedro,Gonzalez,2.0,5.0,NaN,6,4,33.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,11Sep22 ARP,3.0,Travis,Wales,3.0,2.0,NaN,4,5,1.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pdf_df.to_csv("ARAP.csv")